# Get directory

In [10]:
from os import listdir
from os.path import isfile, join

page_path = 'occ_assets/pages'
pages_files = [f for f in listdir(page_path) if isfile(join(page_path, f))]

components_path = 'occ_assets/components'
assets_components_name = [f for f in listdir(components_path)]

assets_components_path = [components_path + '/' +
                          name for name in assets_components_name]

assets_components = zip(assets_components_name, assets_components_path)

components_files = [{'name': name, 'path': path, 'files': [f for f in listdir(
    path) if isfile(join(path, f))]} for name, path in assets_components]


### Build component tree

In [11]:
import json

def get_file(path):
    with open(path) as f:
        d = json.load(f)
    return d


def locate_files_index_layout(file_name):
    folder_compo = [f for f in components_files if f['name'] == file_name][0]
    files = {}
    if 'index.json' in folder_compo['files']:
        files.update(
            {'index': get_file(folder_compo['path']+'/'+'index.json')})

    if 'layout.json' in folder_compo['files']:
        files.update(
            {'layout': get_file(folder_compo['path']+'/'+'layout.json')})
    return files


def get_components_layout(layout):
    componets = []
    for compo in layout:
        if 'components' in compo:
            componets.extend(compo['components'])
    return componets


def get_components(compo_name):
    components_object = {}
    for c_name in compo_name:
        try :
            files = locate_files_index_layout(c_name)
            components_object.update({
                c_name: {
                    'type': files['index']['type'] if 'index' in files else {},
                    'COMPONENTS': get_components(get_components_layout(files['layout']['regions'])) if 'layout' in files else {},
                }
            })
        except Exception as ex:
            print(ex)
    return components_object


file_json_tree = {}
for file in pages_files:
    with open(page_path + '/' + file) as f:
        d = json.load(f)
        compo_name = get_components_layout(d['layout'])
        file_json_tree.update({
            file: {
                'address': d['address'],
                'title': d['title'],
                'COMPONENTS': get_components(compo_name)
            }
        })


### Export asset component tree in json

In [12]:
import json

with open ('export/tree_components.json', 'w', encoding='utf-8') as f:
    json.dump(file_json_tree, f, ensure_ascii=False)

### Build unused compenents

In [13]:
def get_subcomponent(components):
  list_name_components = list(components.keys())

  list_components = list(components.values())
  if(len(list_components) > 0):
    for component in list_components:
      list_name_components += get_subcomponent(component['COMPONENTS'])

  return list_name_components

list_page = list(file_json_tree.values())
list_components = []

for components in list_page:
  list_components += get_subcomponent(components['COMPONENTS'])

list_components = list(set(list_components))

file_json_not_user = {
  "components": [i for i in assets_components_name if i not in list_components]
}


### Export unused components

In [14]:
import json

with open ('export/unused_components.json', 'w', encoding='utf-8') as f:
    json.dump(file_json_not_user, f, ensure_ascii=False)